In [22]:
def highwaylist():
    alph,bet,gam,pro = [],[],[],[]
    with open('alph.txt') as ip1, open('bet.txt') as ip2, open('gam.txt') as ip3, open('pro.txt') as ip4:
        for i1,i2,i3,i4 in zip(ip1,ip2,ip3,ip4):
            hw.append(NMTS(int(i1.strip()),int(i2.strip()),int(i3.strip()),float(i4.strip()))) 


In [14]:
import math
def bias(alpha):
    SBox= [[0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  
          [0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  1,  0,  1],  
          [0,  0,  2,  2,  2,  2,  0,  0,  2,  2,  0,  1,  0,  1,  2,  2],  
          [0,  0,  2,  2,  2,  2,  1,  0,  2,  2,  1,  0,  0,  0,  2,  2],  
          [0,  0,  2,  2,  2,  2,  0,  1,  2,  2,  0,  1,  0,  0,  2,  2],  
          [0,  0,  2,  2,  2,  2,  0,  0,  2,  2,  1,  0,  1,  0,  2,  2],  
          [0,  0,  0,  1,  0,  0,  1,  0,  0,  1,  0,  0,  1,  0,  0,  0],  
          [0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0],  
          [0,  0,  2,  2,  0,  0,  2,  2,  2,  2,  0,  0,  2,  2,  1,  1],  
          [0,  1,  2,  2,  0,  0,  2,  2,  2,  2,  1,  0,  2,  2,  0,  0],  
          [0,  0,  1,  0,  2,  2,  2,  2,  0,  0,  0,  1,  2,  2,  2,  2],  
          [0,  1,  0,  0,  2,  2,  2,  2,  1,  0,  0,  0,  2,  2,  2,  2],  
          [0,  0,  0,  0,  2,  2,  2,  2,  1,  0,  0,  1,  2,  2,  2,  2],  
          [0,  1,  1,  0,  2,  2,  2,  2,  0,  0,  0,  0,  2,  2,  2,  2],  
          [0,  0,  2,  2,  1,  1,  2,  2,  2,  2,  0,  0,  2,  2,  0,  0], 
          [0,  1,  2,  2,  0,  0,  2,  2,  2,  2,  1,  0,  2,  2,  0,  0]] 
    
    
    beta= (random.randint(0,15))
    if(alpha==0):
        return 0, 0
    else:
        while(SBox[alpha][beta]==0):
            beta= (random.randint(1,15))
            #print(SBox[alpha][beta])    
        return SBox[alpha][beta], beta

#print("A[0][0] =", SBox[3][1])   # 3rd element of 3,nd row


In [15]:
PBox = [12, 1, 6, 11, 8, 13, 2, 7, 4, 9, 14, 3, 0, 5, 10, 15]

In [16]:
def sBoxLayer(state):
        """SBox function for encryption

        Input:  16-bit integer
        Output: 16-bit integer"""
        sboxcount=0
        prob=0
        output = 0
        #print('\n')
        for i in range(4):
            inp= (state >> (i*4)) & 0xF
            if(inp!=0):
                sboxcount=sboxcount+1
            p, op= bias(inp)
            prob=prob+p
            #print(p,prob)
            output += op << (i*4)
        return output,prob,sboxcount

In [17]:
def pLayer(state):
        """Permutation layer for encryption

        Input:  16-bit integer
        Output: 16-bit integer"""
        output = 0
        for i in range(16):
                output += ((state >> i) & 0x01) << PBox[i]
        return output

In [18]:
#right circular shifts
def ROR(x,r):
    x = ((x << (CIPHER_TYPE - r)) + (x >> r)) & mask
    return x

#left circular shifts
def ROL(x,r):
    x = ((x >> (CIPHER_TYPE - r)) + (x << r)) & mask
    return x

In [19]:
def find_diff_path(st1,st0,CIPHER_ROUNDS, n):
            lp=1             
            temp_wt=0
            tempwt_list=[]
            tempdec_list=[]
            for i in range (0,CIPHER_ROUNDS):
                op,prob,sbox=sBoxLayer(st0)
                A=pLayer(op)
                B=st1^A
                #addition of random output decision in decision list, if output hold with some differential probability
                tempdec_list.append(NMTS(st1,st0,op,prob,sbox)) 
                #print(tempdec_list[i].prob)
                temp_wt= temp_wt+tempdec_list[i].prob
                #update state with new valid output differential
                st1=st0
                st0=A
            n=n+1
            return tempdec_list, temp_wt, n

In [20]:
def find_best_path(st1,st0,CIPHER_ROUNDS,wt_above,best_wt):
    #print(hex(st1),hex(st0),'best_path')
    #using n as index value for list
    n=0
    for r in range(CIPHER_ROUNDS,0,-1):
        tempdec_list, temp_wt, n = find_diff_path(st1,st0,r, n)
        if((temp_wt+wt_above) < best_wt):
            best_wt= temp_wt+wt_above
            for i,j in zip(range(n-1,25), range(len(tempdec_list))):
                dec_list[i]=(NMTS(tempdec_list[j].da,tempdec_list[j].db,tempdec_list[j].op, tempdec_list[j].prob, tempdec_list[j].sbox))
        #print(n)
        if(n<CIPHER_ROUNDS):
            st1=dec_list[n].da
            st0=dec_list[n].db
        wt_above= wt_above+dec_list[n-1].prob
    return best_wt

In [24]:
#alpha beta are for left and right circular shift  
import random
import math 
CIPHER_ROUNDS=11
CIPHER_TYPE=16
mask = 2 ** 16 - 1
wshift=15
dec_list=[0]*40
wt_above=0
best_wt=999999
s=9999
class NMTS(object):
    """__init__() functions as the class constructor"""
    def __init__(self, da=None, db=None, op=None, prob=None, sbox=None):
        self.da = da
        self.db = db
        self.op = op
        self.prob = prob
        self.sbox = sbox
        
hw=[]
#nested loop to run number of time
highwaylist()
ch=0 
while(best_wt>31):
    ch=hw[ch].da
    ch1=hw[ch].db
    #ch=(random.randint(1,mask))
    #ch1=(random.randint(1,mask))
    #best_wt=find_best_path(0x10000, 0x400000, CIPHER_ROUNDS,wt_above,best_wt)
    best_wt=find_best_path(ch,ch1,CIPHER_ROUNDS,wt_above,best_wt)
    ch=ch+1
    #print(ch)
    if(best_wt<s):
        s=best_wt
        print(s,ch) 

#print(hex(st1),hex(st0),0)
print("Dec list is: st1,st0,op,bias,sbox")   
for i in range(0,CIPHER_ROUNDS): 
    #print("Starting input of %i round and weight:" %i,hex(dec_list[i].da), hex(dec_list[i].db),hex(dec_list[i].dc),hex(dec_list[i].dd),hex(dec_list[i].op1),hex(dec_list[i].op2),hex(dec_list[i].op3),(dec_list[i].wt1),(dec_list[i].wt2),(dec_list[i].wt3),(dec_list[i].wt))
    print(hex(dec_list[i].da), hex(dec_list[i].db), hex(dec_list[i].op), (dec_list[i].prob),(dec_list[i].sbox))
    #print(format(dec_list[i].da, '032b'))
    #print(bin(dec_list[i].da))

print("Best weight is:",best_wt+1) 

54 1
51 32769
50 63233
49 65089
48 65089
47 65089
46 65089
43 65089
42 65089
41 65089
40 65089
34 65089
31 65089
Dec list is: st1,st0,op,bias,sbox
0x0 0x8000 0x6000 2 1
0x8000 0x420 0x420 4 2
0x420 0x6000 0xc000 1 1
0x6000 0x8400 0xfb00 2 2
0x8400 0x8639 0xdc6a 5 4
0x8639 0xec0f 0x5409 5 3
0xec0f 0x5c01 0xa505 4 3
0x5c01 0xd070 0x1040 2 2
0xd070 0x5 0x2 2 1
0x5 0x2 0x4 2 1
0x2 0x40 0x30 2 1
Best weight is: 32
